# 02_eda_visuals — EDA + figures

Ce notebook lit `data/processed/eu_ev_by_country_year.csv`, vérifie l'intégrité, 
et génère des figures prêtes pour le dossier `dashboard/`.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

proc_path = Path("../data/processed/eu_ev_by_country_year.csv")
dash = Path("../dashboard"); dash.mkdir(exist_ok=True)

assert proc_path.exists(), f"Fichier manquant: {proc_path}"
df = pd.read_csv(proc_path)

expected_cols = {"country_code", "year", "powertrain", "registrations"}
missing = expected_cols - set(df.columns)
assert not missing, f"Colonnes manquantes: {missing}. Colonnes trouvées: {df.columns.tolist()}"

df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
df['registrations'] = pd.to_numeric(df['registrations'], errors='coerce')
df = df.dropna(subset=['year', 'registrations'])
df['year'] = df['year'].astype(int)

display(df.head())
print('Lignes totales:', len(df))
print('Années:', sorted(df['year'].unique()))
print('Powertrains:', sorted(df['powertrain'].unique()))
print('Ex pays:', sorted(df['country_code'].unique())[:10])


## Sanity checks rapides
- Total par année et motorisation
- Détection de séries vides (qui expliqueraient des courbes invisibles)


In [ ]:
pivot_check = (
    df.groupby(['year','powertrain'], as_index=False)['registrations'].sum()
    .pivot(index='year', columns='powertrain', values='registrations')
    .fillna(0)
)
display(pivot_check)
empty_cols = [c for c in pivot_check.columns if (pivot_check[c] == 0).all()]
if empty_cols:
    print('⚠️ Motorisations sans données sur toutes les années:', empty_cols)
else:
    print('OK: toutes les motorisations ont au moins une valeur > 0')


## Figure A — Top 5 pays EV (BEV+PHEV) en 2023


In [ ]:
y = 2023
subset = df[(df['year']==y) & (df['powertrain'].isin(['BEV','PHEV']))]
agg = subset.groupby('country_code', as_index=False)['registrations'].sum()
agg = agg.sort_values('registrations', ascending=False).head(5)

if agg.empty:
    print('⚠️ Aucune donnée BEV/PHEV pour', y)
else:
    plt.figure()
    plt.bar(agg['country_code'], agg['registrations'])
    plt.title(f'Top 5 pays EV (BEV+PHEV) – {y}')
    plt.xlabel('Pays')
    plt.ylabel('Nouvelles immatriculations')
    plt.tight_layout()
    plt.savefig(dash / f'top5_countries_ev_{y}.png', dpi=150)
    plt.show()


## Figure B — Évolution UE 2015–2023 : BEV vs PHEV


In [ ]:
eu_trend = (
    df[df['powertrain'].isin(['BEV','PHEV'])]
    .groupby(['year','powertrain'], as_index=False)['registrations'].sum()
)
if eu_trend.empty:
    print('⚠️ Pas de données BEV/PHEV pour tracer la tendance UE')
else:
    plt.figure()
    for p in ['BEV','PHEV']:
        sub = eu_trend[eu_trend['powertrain']==p]
        plt.plot(sub['year'], sub['registrations'], marker='o', label=p)
    plt.legend()
    plt.title('Évolution UE – BEV vs PHEV (2015–2023)')
    plt.xlabel('Année')
    plt.ylabel('Nouvelles immatriculations')
    plt.tight_layout()
    plt.savefig(dash / 'eu_trend_bev_phev.png', dpi=150)
    plt.show()


## Figure C — Mix motorisations UE en 2023 (camembert)


In [ ]:
mix_2023 = (
    df[df['year']==y]
    .groupby('powertrain', as_index=False)['registrations'].sum()
    .sort_values('registrations', ascending=False)
)
if mix_2023.empty:
    print('⚠️ Pas de données pour le camembert', y)
else:
    plt.figure()
    plt.pie(mix_2023['registrations'], labels=mix_2023['powertrain'], autopct='%1.1f%%')
    plt.title(f'Mix motorisations – UE {y}')
    plt.tight_layout()
    plt.savefig(dash / f'mix_powertrain_{y}.png', dpi=150)
    plt.show()


## Figure D — Tendances BEV par pays (sélection)


In [ ]:
focus = ['DE','FR','NL','UK','NO','SE','IT','ES']
bev = df[(df['powertrain']=='BEV') & (df['country_code'].isin(focus))]
if bev.empty:
    print('⚠️ Pas de données BEV pour les pays focus')
else:
    plt.figure()
    for cc in sorted(bev['country_code'].unique()):
        sub = bev[bev['country_code']==cc].sort_values('year')
        plt.plot(sub['year'], sub['registrations'], marker='o', label=cc)
    plt.legend(title='Pays', ncol=4)
    plt.title('Tendances BEV par pays (sélection)')
    plt.xlabel('Année')
    plt.ylabel('Nouvelles immatriculations')
    plt.tight_layout()
    plt.savefig(dash / 'bev_trends_focus.png', dpi=150)
    plt.show()
